Soit un restaurant :

Le restaurant dispose de 20 chaises pour ses clients.
D'après des statistiques sur 100 heures le restaurant a reçu 900 clients.
Le restaurant dispose d'un seul cuisinier qui traite chaque commande dans une durée moyenne de 10 minutes.
Le temps moyen pour qu'un client consomme sa commande est de 10 minutes.
Chaque client occupe une chaise à partir du moment de son arrivée + le temps d’attente de sa commande + le temps de consommation de sa commande, un client est rejeté si toutes les chaises sont occupées lors de son arrivée.
Q1. Ecrire un programme qui permet de :

Simuler par minute l’arrivée des clients au restaurant, stockez dans une liste le moment (minute) d’arrivée de chaque client.
Vérifiez s'il y a une chaise vide pour chaque client qui arrive, si oui, incrémentez le nombre de chaises occupées puis mettre en file d’attente la commande du client pour être traitée par le cuisinier, sinon s'il n'y a pas de chaise libre le client est rejeté.
Simulez dans le même programme la durée de traitement de chaque commande de la file d’attente qui se fait par le cuisinier par la politique premier arrivé premier servi, stockez dans une liste le moment de fin de traitement de chaque commande.
Simulez dans le même programme la durée de consommation de chaque client lorsque sa commande est servie, puis décrémentez le nombre de chaises occupées afin de libérer une chaise.
Q2. Faites une simulation sur 100 heures puis estimez :

Le nombre total de clients reçus
Le nombre de clients rejetés
Le temps moyen que passe un client à attendre sa commande
Que pensez vous de ces chiffres ?

Q3.

Si on augmente la capacité du restaurant à 40 chaises, peut-on améliorer les métriques calculées en Q2? (testez par une simulation sur 100 heures)
En ajoutant un 2ième cuisinier le temps moyen de traitement d'une commande passe à 5 minutes, est-ce que cette solution permet d'améliorer les métriques calculées en Q2 ? (testez par une simulation sur 100 heures)

source : Exercice 3 fiche TP1 MODSIM M1 - USTOMB

In [1]:
import numpy as np
import numpy.random as rd

#Fonction de simulation de l'arrivée d'un événement à partir de la probabilité de son arrivée
# événement arrivé si retour=true, p: probabilité d'arrivée
def sim_proba(p): 
    return rd.choice([True, False],p=[p, 1-p])

# fonction de simulation
# p: probabilité d'arrivée d'un client
# p_traitement: probabilité de fin du traitement d'une commande
# duree: durée de la simulation
# chaisesLib: nombre de chaises libres au debut de la simulation
# afficher_simulation: vrai si on veux afficher les details de la simulation, faux sinon
def simulation_clients(p, p_traitement, duree, chaisesLib, afficher_simulation):
    
    moment_arrivee=[] # liste pour stocker le moment d'arrivée de chaque client
    moment_fin_traitement=[] # liste pour stocker le moment de fin de traitement de chaque commande
    commandes = [] # file d'attente des commandes
    consommation = [] # liste des consommations en cours
    rejet = 0 # nombre des clients rejetés  
    id_client=0  #numéro du client
    
    print("--------------------------------------- début de la simulation ---------------------------------------------")
    
    for t in range(duree):

        # Q1.1&2.Simuler l'arrivée d'un client
        
        if sim_proba(p): #Arrivée d'un client si retour=True,  p: probabilité d'arrivée
            id_client=id_client+1 #passer à ce client
            
            #s'il ne reste pas de chaises libres --> rejeter le client
            if chaisesLib <= 0 : 
                if afficher_simulation : print("Minute %d: reject du client%d"%(t, id_client))
                rejet += 1
                
            else : # au moins une chaise est libre
                if afficher_simulation : print("Minute %d: arrivée du client%d"%(t, id_client))
                moment_arrivee.append (t) # stocker le moment d'arrivée de chaque client
                chaisesLib -= 1 #occuper une chaise
                commandes.append(id_client) #faire une commande
        
        # Q1.3.Simuler la préparation d'une commande
        
        if len(commandes) > 0 : # au moins une commande est en attente
            if sim_proba(p_traitement): #une commande est prete si retour=True, p_traitement: #probabilité du fin de traitement
                client = commandes.pop(0) # servir la premiere commande arrivée
                if afficher_simulation : print("Minute %d: le cuisinier a preparé la commande du client%d"%(t,client))
                moment_fin_traitement.append (t) #stocker le moment de fin de traitement de chaque commande
                consommation.append(client) # commencer la consommation de la commande par le client
                
        # Q1.4.Simuler la consommation d'une commande         
                
        if len(consommation) > 0 : # au moins une consommation est en cours
            if sim_proba(0.1): #client à terminé sa consommation si retour=True, 0.1: temps moyen de consommation=10min
                client = consommation.pop(0) # terminer une consommation
                if afficher_simulation : print("Minute %d: le client%d a terminé sa consommation"%(t,client))
                chaisesLib=chaisesLib+1 # libérer la chaise
                
    print("---------------------------------------- fin de la simulation ------------------------------------------\n")
    
    # Q2.1&2&3
    print ("nombre des clients reçus : ",id_client - 1)
    print ("nombre des clients rejetés : ",rejet)
    
    temps_attente = 0
    for i in range (len(moment_fin_traitement)) :
        # il reste des commandes en attente à la fin de la simulation : len(moment_arrivee) >= len(moment_fin_traitement)
        temps_attente = temps_attente + (moment_fin_traitement[i] - moment_arrivee[i])
    print ("temps attente moyen : ", temps_attente/len(moment_fin_traitement)) 
    
print("\n############################# Appel de la simulation pour 100 heures ###################################")

afficher_simulation = True # si vrai afficher la simulation + metriques / sinon afficher uniquement les metriques

p_arrive=0.15  # la probabilité d'arrivée d'un client par minute 900/100/60=0.15
p_traitement = 0.1 #probabilité du fin de traitement d'une commande par minute 1/10=0.1
duree=100*60    # durée de la simulation: 2heures x 60minutes
chaisesLib = 20 #nombre de chaises libres  
simulation_clients(p_arrive, p_traitement, duree, chaisesLib, afficher_simulation)

print ("\n Que pensez vous de ces chiffres ?")
print("beaucoup de clients rejetes et temps d'attente tres long, possibilité d'amelioration\n plus la durée de simulation est longue plus ces resultats sont proches des moyennes calculés")

# Q3.1.
print("\npeut-on améliorer les métriques calculées en Q2 avec 40 chaises ?")
print("On peut diminuer le nombre des clients rejetés au debut mais le temps d'attente serait tres long\n et eventuellement on commencera a rejeter les clients de nouveau")

print("\n############################ Appel de la simulation avec 40 chaise pour 100 heures ###################")
afficher_simulation = False
duree=100*60    # durée de la simulation: 100heures x 60minutes
chaisesLib = 40 #doubler le nombre de chaises libres
simulation_clients(p_arrive, p_traitement, duree, chaisesLib, afficher_simulation)

# Q3.2.
print("\nest-ce que l'ajout d'un deuxieme cuisinier permet d'améliorer les métriques calculées en Q2 ?")
print("Oui, ça permet de diminuer le temps d'attente du traitement de la commande\n et donc le nombre des clients rejetés aussi")

print("\n########################### Appel de la simulation avec 2 cuisiniers pour 100 heures ##################")
afficher_simulation = False
p_traitement = 0.2 #probabilité du fin de traitement d'une commande par minute 1/5=0.2
simulation_clients(p_arrive, p_traitement, duree, chaisesLib,afficher_simulation)


############################# Appel de la simulation pour 100 heures ###################################
--------------------------------------- début de la simulation ---------------------------------------------
Minute 1: arrivée du client1
Minute 7: arrivée du client2
Minute 9: arrivée du client3
Minute 11: arrivée du client4
Minute 14: le cuisinier a preparé la commande du client1
Minute 15: arrivée du client5
Minute 15: le client1 a terminé sa consommation
Minute 16: arrivée du client6
Minute 17: le cuisinier a preparé la commande du client2
Minute 29: le cuisinier a preparé la commande du client3
Minute 30: arrivée du client7
Minute 31: le client2 a terminé sa consommation
Minute 32: arrivée du client8
Minute 33: le cuisinier a preparé la commande du client4
Minute 42: arrivée du client9
Minute 43: le cuisinier a preparé la commande du client5
Minute 43: le client3 a terminé sa consommation
Minute 45: le client4 a terminé sa consommation
Minute 50: arrivée du client10
Minute 53:

Minute 1214: arrivée du client183
Minute 1215: reject du client184
Minute 1217: reject du client185
Minute 1217: le client135 a terminé sa consommation
Minute 1220: arrivée du client186
Minute 1225: reject du client187
Minute 1238: le client136 a terminé sa consommation
Minute 1239: le client145 a terminé sa consommation
Minute 1240: arrivée du client188
Minute 1240: le client147 a terminé sa consommation
Minute 1246: le client148 a terminé sa consommation
Minute 1252: le client149 a terminé sa consommation
Minute 1254: arrivée du client189
Minute 1254: le cuisinier a preparé la commande du client166
Minute 1254: le client150 a terminé sa consommation
Minute 1256: arrivée du client190
Minute 1260: le client152 a terminé sa consommation
Minute 1261: arrivée du client191
Minute 1266: arrivée du client192
Minute 1272: arrivée du client193
Minute 1272: le cuisinier a preparé la commande du client171
Minute 1273: le client158 a terminé sa consommation
Minute 1275: le cuisinier a preparé la 

Minute 2272: arrivée du client334
Minute 2284: arrivée du client335
Minute 2285: le client307 a terminé sa consommation
Minute 2293: arrivée du client336
Minute 2294: arrivée du client337
Minute 2298: arrivée du client338
Minute 2305: reject du client339
Minute 2306: le client308 a terminé sa consommation
Minute 2317: le cuisinier a preparé la commande du client332
Minute 2318: arrivée du client340
Minute 2319: le cuisinier a preparé la commande du client333
Minute 2321: reject du client341
Minute 2321: le cuisinier a preparé la commande du client334
Minute 2324: reject du client342
Minute 2327: le cuisinier a preparé la commande du client335
Minute 2328: le cuisinier a preparé la commande du client336
Minute 2331: le client310 a terminé sa consommation
Minute 2332: arrivée du client343
Minute 2338: le cuisinier a preparé la commande du client337
Minute 2343: le client311 a terminé sa consommation
Minute 2347: le client312 a terminé sa consommation
Minute 2354: arrivée du client344
Min

Minute 3676: arrivée du client545
Minute 3677: reject du client546
Minute 3679: reject du client547
Minute 3680: reject du client548
Minute 3697: reject du client549
Minute 3699: reject du client550
Minute 3699: le cuisinier a preparé la commande du client521
Minute 3702: le client517 a terminé sa consommation
Minute 3705: le cuisinier a preparé la commande du client522
Minute 3707: arrivée du client551
Minute 3708: le client518 a terminé sa consommation
Minute 3709: arrivée du client552
Minute 3710: reject du client553
Minute 3714: reject du client554
Minute 3719: reject du client555
Minute 3720: le cuisinier a preparé la commande du client523
Minute 3721: le client519 a terminé sa consommation
Minute 3722: le client520 a terminé sa consommation
Minute 3725: arrivée du client556
Minute 3727: arrivée du client557
Minute 3727: le client521 a terminé sa consommation
Minute 3730: le cuisinier a preparé la commande du client524
Minute 3731: le client522 a terminé sa consommation
Minute 373

Minute 4914: le client711 a terminé sa consommation
Minute 4920: le cuisinier a preparé la commande du client712
Minute 4923: le client712 a terminé sa consommation
Minute 4924: arrivée du client740
Minute 4925: le cuisinier a preparé la commande du client713
Minute 4926: arrivée du client741
Minute 4929: le client713 a terminé sa consommation
Minute 4936: arrivée du client742
Minute 4945: reject du client743
Minute 4946: reject du client744
Minute 4950: reject du client745
Minute 4950: le cuisinier a preparé la commande du client714
Minute 4951: le cuisinier a preparé la commande du client715
Minute 4952: le cuisinier a preparé la commande du client716
Minute 4954: reject du client746
Minute 4954: le cuisinier a preparé la commande du client717
Minute 4954: le client714 a terminé sa consommation
Minute 4960: le client715 a terminé sa consommation
Minute 4961: arrivée du client747
Minute 4964: le client716 a terminé sa consommation
Minute 4966: le client717 a terminé sa consommation
Mi

---------------------------------------- fin de la simulation ------------------------------------------

nombre des clients reçus :  941
nombre des clients rejetés :  344
temps attente moyen :  232.98625429553263

est-ce que l'ajout d'un deuxieme cuisinier permet d'améliorer les métriques calculées en Q2 ?
Oui, ça permet de diminuer le temps d'attente du traitement de la commande
 et donc le nombre des clients rejetés aussi

########################### Appel de la simulation avec 2 cuisiniers pour 100 heures ##################
--------------------------------------- début de la simulation ---------------------------------------------
---------------------------------------- fin de la simulation ------------------------------------------

nombre des clients reçus :  920
nombre des clients rejetés :  310
temps attente moyen :  9.451718494271686
